In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [3]:
#spacy for lemmatization
import spacy
from pprint import pprint

In [4]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/root1/Documents/lda/myvenv/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# NLTK for stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['tagum', 'city', 'ordinance', 'municipal', 'municipality', 'thereof'])

In [7]:
# Import Dataset
df = pd.read_csv('tagum_ordinances.csv', header = 0)
print(df)
df.head()

                                     OR  \
0        CITY ORDINANCE No. 842, s-2018   
1        CITY ORDINANCE NO. 825, s-2017   
2        CITY ORDINANCE NO. 837, s-2018   
3        CITY ORDINANCE NO. 844, s-2018   
4        CITY ORDINANCE NO. 838, s-2018   
..                                  ...   
670  MUNICIPAL ORDINANCE NO. 05, s-1974   
671  MUNICIPAL ORDINANCE NO. 04, s-1974   
672  MUNICIPAL ORDINANCE NO. 01, s-1974   
673    MUNICIPAL ORDINANCE NO. 03, 1973   
674  MUNICIPAL ORDINANCE NO. 02, S-1973   

                                                    OT  
0    “AN ORDINANCE REGULATING THE PRACTICE OF TRADI...  
1    “AN ORDINANCE RECONSTITUTING THE MEMBERS OF TH...  
2    “AN ORDINANCE ESTABLISHING FUNERAL AND BURIAL ...  
3    AN ORDINANCE AMENDING THE TITLE, SECTION 1, SE...  
4    “AN ORDINANCE CREATING THE CITY HISTORICAL, CU...  
..                                                 ...  
670  “AN ORDINANCE AMENDING SECTION 10 OF MUNICIPAL...  
671  “AN ORDINANCE REGULATI

,OR,OT
0,"CITY ORDINANCE No. 842, s-2018",“AN ORDINANCE REGULATING THE PRACTICE OF TRADI...
1,"CITY ORDINANCE NO. 825, s-2017",“AN ORDINANCE RECONSTITUTING THE MEMBERS OF TH...
2,"CITY ORDINANCE NO. 837, s-2018",“AN ORDINANCE ESTABLISHING FUNERAL AND BURIAL ...
3,"CITY ORDINANCE NO. 844, s-2018","AN ORDINANCE AMENDING THE TITLE, SECTION 1, SE..."
4,"CITY ORDINANCE NO. 838, s-2018","“AN ORDINANCE CREATING THE CITY HISTORICAL, CU..."


In [8]:
# List all ordinance titles
# Convert to list
data = list(df.OT) 
print(data[:1])

['“AN ORDINANCE REGULATING THE PRACTICE OF TRADITIONAL HOME BIRTH DELIVERY OR DELIVERIES ATTENDED BY A TRADITIONAL BIRTH ATTENDANT OR MANANABANG”.']


In [21]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\"", "", sent) for sent in data]

# Remove words ending with ING like amending, regulating
data = [re.sub('\S*ING\S*\s?', '', sent) for sent in data]


pprint(data[:5])

['“AN ORDINANCE THE PRACTICE OF TRADITIONAL HOME BIRTH DELIVERY OR DELIVERIES '
 'ATTENDED BY A TRADITIONAL BIRTH ATTENDANT OR MANANABANG”.',
 '“AN ORDINANCE THE MEMBERS OF THE CITY TRICYCLE AND REGULATORY BOARD (CTFRB) '
 'ITS FUNCTIONS AND PROCEDURES AND MOTORIZED TRICYCLE FOR HIRE (MTH), TRICYCLE '
 'UTILITY VEHICLE (TUV) AND TRICYCLE UTILITY FOR HIRE (TUH) GUIDELINES, '
 'REGULATIONS AND OPERATIONS, THE COLLECTION OF FEES, CHARGES,ADMINISTRATIVE '
 'PROVISIONS AND PENALTIES FOR VIOLATION THEREOF”.',
 '“AN ORDINANCE FUNERAL AND BURIAL ASSISTANCE FOR INDIGENT TAGUMENYOS, ITS '
 'MANAGEMENT, PROCEDURES AND GUIDELINES AND FUNDS THEREFOR”.',
 'AN ORDINANCE THE TITLE, SECTION 1, SECTION 2 AND SECTION 3 OF CITY ORDINANCE '
 'NO. 820, S-2017 “AN ORDINANCE INTO A LOAN AGREEMENT WITH THE DEVELOPMENT '
 'BANK OF THE PHILIPPINES, IN THE AMOUNT OF SIX HUNDRED EIGHTY MILLION PESOS '
 '(P 680,000,000.00) TO FUND THE ACQUISITION OF HEAVY EQUIPMENT,ACQUISITION OF '
 'SERVICE VEHICLES, ACQUISITION O

In [23]:
#preprocess using gensim
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:5])

[['an', 'ordinance', 'the', 'practice', 'of', 'traditional', 'home', 'birth', 'delivery', 'or', 'deliveries', 'attended', 'by', 'traditional', 'birth', 'attendant', 'or', 'mananabang'], ['an', 'ordinance', 'the', 'members', 'of', 'the', 'city', 'tricycle', 'and', 'regulatory', 'board', 'ctfrb', 'its', 'functions', 'and', 'procedures', 'and', 'motorized', 'tricycle', 'for', 'hire', 'mth', 'tricycle', 'utility', 'vehicle', 'tuv', 'and', 'tricycle', 'utility', 'for', 'hire', 'tuh', 'guidelines', 'regulations', 'and', 'operations', 'the', 'collection', 'of', 'fees', 'charges', 'administrative', 'provisions', 'and', 'penalties', 'for', 'violation', 'thereof'], ['an', 'ordinance', 'funeral', 'and', 'burial', 'assistance', 'for', 'indigent', 'tagumenyos', 'its', 'management', 'procedures', 'and', 'guidelines', 'and', 'funds', 'therefor'], ['an', 'ordinance', 'the', 'title', 'section', 'section', 'and', 'section', 'of', 'city', 'ordinance', 'no', 'an', 'ordinance', 'into', 'loan', 'agreement',

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['an', 'ordinance', 'the', 'practice', 'of', 'traditional', 'home', 'birth', 'delivery', 'or', 'deliveries', 'attended', 'by', 'traditional', 'birth', 'attendant', 'or', 'mananabang']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

pprint(data_lemmatized[:4])

[['practice',
  'traditional',
  'home',
  'birth',
  'delivery',
  'delivery',
  'attend',
  'traditional',
  'birth',
  'attendant',
  'mananabang'],
 ['function',
  'procedure',
  'tricycle',
  'utility',
  'vehicle',
  'tricycle',
  'utility',
  'hire',
  'guideline',
  'operation',
  'collection',
  'fee',
  'charge',
  'administrative',
  'provision',
  'penalty',
  'violation'],
 ['indigent', 'management', 'procedure', 'guideline', 'fund'],
 ['section',
  'loan',
  'amount',
  'acquisition',
  'equipment',
  'acquisition',
  'service',
  'vehicle',
  'acquisition',
  'continue_reading']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 2)]]


In [15]:
id2word[0]

'attend'

In [16]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('attend', 1),
  ('attendant', 1),
  ('birth', 2),
  ('delivery', 2),
  ('home', 1),
  ('mananabang', 1),
  ('practice', 1),
  ('traditional', 2)]]

In [50]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [51]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.078*"penalty" + 0.069*"violation" + 0.050*"public" + 0.024*"vehicle" + 0.020*"market" + 0.018*"government" + 0.018*"office" + 0.017*"fee" + 0.017*"use" + 0.015*"road"'), (1, '0.026*"service" + 0.024*"mayor" + 0.022*"amount" + 0.019*"fee" + 0.017*"permit" + 0.016*"private" + 0.015*"development" + 0.014*"house" + 0.011*"appropriation" + 0.011*"annual"'), (2, '0.037*"position" + 0.025*"fund" + 0.023*"necessary" + 0.021*"legislative" + 0.018*"compensation" + 0.018*"market" + 0.016*"officer" + 0.014*"situate" + 0.012*"staff" + 0.012*"establishment"'), (3, '0.031*"regulation" + 0.024*"street" + 0.021*"business" + 0.019*"medical" + 0.019*"guideline" + 0.018*"new" + 0.018*"rule" + 0.018*"implementation" + 0.016*"portion" + 0.015*"certain"'), (4, '0.064*"section" + 0.059*"purpose" + 0.043*"fund" + 0.031*"area" + 0.025*"duty" + 0.022*"tax" + 0.020*"operation" + 0.019*"provide" + 0.018*"establishment" + 0.016*"year"')]


In [52]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.761623310096045

Coherence Score:  0.49439785806596276


In [53]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.274419 -0.133751       1        1  28.676291
4      0.181540 -0.180842       2        1  21.978090
3      0.092216  0.035113       3        1  18.760288
1     -0.057280  0.202265       4        1  15.680583
2      0.057943  0.077215       5        1  14.904748, topic_info=              Term        Freq       Total Category  logprob  loglift
16         penalty  110.000000  110.000000  Default  30.0000  30.0000
22       violation   98.000000   98.000000  Default  29.0000  29.0000
31         section   70.000000   70.000000  Default  28.0000  28.0000
75         purpose   67.000000   67.000000  Default  27.0000  27.0000
144         public   70.000000   70.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
23            fund   17.980775   73.050166   Topic5  -3.7077   0.5016
365         market   12.960148   41.862617   Topic5  -4.0351   0.7310
170          local    6.897465   14.320512   Topic5  -4.6658   1.1730
123  establishment    8.943947   28.614627   Topic5  -4.4060   0.7405
348           peso    6.607933   13.596512   Topic5  -4.7087   1.1819

[215 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
80        3  0.950996            act
821       5  0.941292  administrator
484       2  0.953174   agricultural
374       5  0.832665      allowance
203       4  0.902554          amend
...     ...       ...            ...
799       3  0.887218      vehicular
22        1  0.997461      violation
77        3  0.923708         worker
55        2  0.853439           year
55        4  0.100405           year

[198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 2, 3])

In [54]:
mallet_path = '/Users/root1/Documents/lda/mallet/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command '/Users/root1/Documents/lda/mallet/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /var/folders/7k/y671ljdx4ms4bc3_h8csf0340000gn/T/8ca473_corpus.txt --output /var/folders/7k/y671ljdx4ms4bc3_h8csf0340000gn/T/8ca473_corpus.mallet' returned non-zero exit status 1.